## Выбор подходящих баз данных


## Глоссарий: ACID
Требования ACID обеспечивают сохранность данных в транзакционных системах:
*   **A (Atomicity) — Атомарность**: Транзакция либо выполняется целиком, либо не выполняется вовсе.
*   **C (Consistency) — Согласованность**: Транзакция переводит базу из одного согласованного состояния в другое.
*   **I (Isolation) — Изолированность**: Параллельные транзакции не влияют друг на друга.
*   **D (Durability) — Надёжность**: Подтвержденные изменения не потеряются даже при сбое питания.

---

## Основные виды БД
1.  **Реляционные (RDBMS)**: Таблицы, связи, SQL, ACID. (PostgreSQL, MySQL).
2.  **Key-Value**: Простые пары ключ-значение, быстрые, легко масштабируются. (Redis, DynamoDB).
3.  **Колоночные (Columnar)**: Хранят данные по колонкам. Идеальны для аналитики (OLAP) и агрегаций. (Cassandra, ClickHouse, HBase).
4.  **Документные**: Хранят JSON-подобные документы. Гибкая схема. (MongoDB, CouchDB).

---

## CAP и PACELC теоремы
### CAP Теорема
В распределенной системе можно гарантировать только 2 из 3 свойств:
1.  **C (Consistency)**: Согласованность (все видят одни и те же данные).
2.  **A (Availability)**: Доступность (система отвечает на запросы).
3.  **P (Partition Tolerance)**: Устойчивость к разделению сети.

Так как в распределенных системах P неизбежно (сеть может моргнуть), мы выбираем между **CP** (система падает, но не врет) и **AP** (система работает, но может отдать устаревшие данные).

### PACELC Теорема
Расширение CAP:
*   Если есть Partition (P), то выбор между A и C.
*   Else (E) (когда сеть работает штатно), выбор между **Latency (L)** и **Consistency (C)**.
*(Хотим ли мы отвечать мгновенно или ждать полной синхронизации реплик?)*

---


## Практический выбор БД для кейсов

### Сокращатель ссылок

**Данные:**
*   Короткий ключ -> Длинная ссылка (Mapping).
*   Опционально: профиль пользователя, статистика.

**Анализ:**
*   Миллиарды записей.
*   Простая структура, JOIN-ы не нужны.
*   ACID не критичен (если ссылка заработает на секунду позже — ок).
*   **Read-heavy** нагрузка.

**Выбор:**
*   **NoSQL (Key-Value или Wide-Column)**.
*   Идеально: **Cassandra** или **DynamoDB** (масштабируемость), **Redis** (для кэширования горячих ссылок).


### Хостинг текстов (Pastebin)

**Данные:**
*   Метаданные текста (ID, автор, дата).
*   Контент (текст).

**Анализ:**
*   Миллиарды записей.
*   JOIN-ы не нужны.
*   Тексты могут быть большими.

**Выбор:**
*   Метаданные: **Cassandra / DynamoDB**.
*   Контент: **Object Storage (S3)** (если тексты большие) или тоже в NoSQL (если лимиты позволяют).


### Облачный диск

**Данные:**
*   Метаданные: юзеры, папки, права, структура файлов.
*   Блоки файлов (Chunks).

**Анализ:**
*   Метаданные требуют **ACID** (важна целостность структуры папок).
*   Блоки файлов просто хранятся по ID.

**Выбор:**
*   Метаданные: **Реляционная БД (PostgreSQL)** с шардированием. Или **NewSQL** (CockroachDB).
*   Файлы: **Object Storage (S3)** (Cold/Hot storage).


### Фотоаппы (Instagram)

**Данные:**
*   Профиль юзера.
*   Связи (подписки/фолловеры).
*   Фото/Видео.
*   Лайки/Комменты.

**Анализ:**
*   Нужны JOIN-ы для построения ленты (SQL удобен).
*   Но объем данных огромен -> SQL сложно скейлить.

**Выбор:**
*   Профили/Связи: **PostgreSQL** (шардированный) или Графовая БД (Neo4j).
*   Медиа: **Object Storage (S3)** + **CDN**.
*   Лента/Лайки: **Cassandra** (быстрая запись) или **Redis** (для кэширования готовой ленты).


### Телеграм

**Данные:**
*   Профиль, контакты.
*   История сообщений (огромный объем).
*   Статус (Online).

**Выбор:**
*   Состояние/Сессии: **Redis**.
*   Контакты/Профиль: **PostgreSQL**.
*   История сообщений: **Cassandra / HBase** (Wide-column отлично подходит для time-series данных чатов).
*   Медиа: **Object Storage**.


### Твиттер

**Данные:**
*   Твиты, Профили, Граф подписок (Social Graph).

**Выбор:**
*   Твиты: **Key-Value / Wide-Column** (нужен ID -> Content).
*   Граф подписок: **Graph DB** или шардированный **MySQL/PostgreSQL** (FlockDB у Twitter).
*   Таймлайны: **Redis** (храним готовые списки ID твитов для ленты пользователя).


### Нетфликс

**Данные:**
*   Каталог фильмов, Актеры.
*   Биллинг/Подписки.
*   Статистика просмотров (Logs/Events).

**Выбор:**
*   Каталог/Биллинг: **PostgreSQL** (нужен ACID и сложная структура).
*   Статистика/Логи: **Cassandra / ClickHouse** (Big Data аналитика).
*   Рекомендации: **Redis** (готовые списки).
*   Видео: **Object Storage + CDN**.


### Сервис такси

**Данные:**
*   Гео-позиции (Geo-spatial data).
*   Транзакции поездок.

**Выбор:**
*   Позиции: **Redis** (Geo-add) или БД с гео-индексом (**PostGIS**, **Mongo**).
*   Поездки/Биллинг: **PostgreSQL** (строгий ACID).
*   Архив поездок: **Cassandra** (история).
